In [1]:
import tensorflow as tf
import numpy as np
import os
import time
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense,Embedding,GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector

## Create mapping functions as the model works with numbers

In [2]:
char2idx = {'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, ',': 5, '-': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20, 'K': 21, 'L': 22, 'M': 23, 'N': 24, 'O': 25, 'P': 26, 'R': 27, 'S': 28, 'T': 29, 'U': 30, 'V': 31, 'W': 32, 'Z': 33, 'a': 34, 'b': 35, 'c': 36, 'd': 37, 'e': 38, 'f': 39, 'g': 40, 'h': 41, 'i': 42, 'j': 43, 'k': 44, 'l': 45, 'm': 46, 'n': 47, 'o': 48, 'p': 49, 'q': 50, 'r': 51, 's': 52, 't': 53, 'u': 54, 'v': 55, 'w': 56, 'x': 57, 'y': 58, 'z': 59, 'Ó': 60, 'à': 61, 'é': 62, 'ó': 63, 'ą': 64, 'Ć': 65, 'ć': 66, 'ę': 67, 'Ł': 68, 'ł': 69, 'ń': 70, 'Ś': 71, 'ś': 72, 'Ź': 73, 'ź': 74, 'Ż': 75, 'ż': 76, '—': 77, '’': 78, '“': 79, '„': 80}
idx2char = ['\n',' ','!','(',')',',','-','.',':',';','?','A','B','C','D','E','F','G',
 'H','I','J','K','L','M','N','O','P','R','S','T','U','V','W','Z','a','b',
 'c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t',
 'u','v','w','x','y','z','Ó','à','é','ó','ą','Ć','ć','ę','Ł','ł','ń','Ś',
 'ś','Ź','ź','Ż','ż','—','’','“','„']

## Load the model

In [3]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
from tensorflow.keras.utils import get_custom_objects
get_custom_objects().update({"loss": loss})
model = tf.keras.models.load_model('model_5800.h5')
#model.save_weights('weights_5800.h5')

In [51]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model = tf.keras.models.load_model("model_5800.h5", custom_objects={'loss': loss})


In [3]:
#However, if you wish to just perform inference with your model and not further optimization or training your model, you can simply wish to ignore the loss function like this:

model = tf.keras.models.load_model('model_5800.h5',compile=False)

In [ ]:
def generate_text(model, start_string, size=1000):
  # Evaluation step (generating text using the learned model)
    print(start_string)
    num_generate = size

    # Convert our start_string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    #print(input_eval.shape)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0
    

    # Here batch size == 64
    model.reset_states()
    for i in range(num_generate):
        print("====================================================")
        print('input_eval',input_eval[0])

        # make the batch...
        input_eval_64 = input_eval
        for i in range(63):
            input_eval_64 = np.vstack((input_eval_64,input_eval))
        input_eval = input_eval_64

        
        predictions = model(input_eval)
        full_predictions = predictions
        print('full_predictions shape',full_predictions.shape)
        
        predictions = predictions[0]
        
        print('predictions shape',predictions.shape)
        print('predictions',predictions)
        print('predictions',to_text(predictions.numpy().argmax(axis=1)))
        # remove the batch dimension
        #predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        x = tf.random.categorical(predictions, num_samples=10)
        print(x.shape)
        print(x)
        for j in range(x.shape[1]):
              print(to_text(x[:,j].numpy()))
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        print('predicted_id',predicted_id)
        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        
        input_eval = tf.expand_dims([predicted_id], 0)
        print(input_eval)
        text_generated.append(idx2char[predicted_id])
        print('text so far',text_generated)
    return (start_string + ''.join(text_generated))

txt = generate_text(model, start_string="Polsko ",size=500)

Polsko 
input_eval tf.Tensor([26 48 45 52 44 48  1], shape=(7,), dtype=int32)
full_predictions shape (64, 7, 81)
predictions shape (7, 81)
predictions tf.Tensor(
[[-5.94416618e-01 -2.07825232e+00 -5.43718672e+00 -2.53891158e+00
  -7.23920679e+00 -4.59389400e+00 -4.71974468e+00 -5.14035177e+00
  -6.09766865e+00 -6.45774221e+00 -7.01796675e+00 -2.03258491e+00
   2.98637390e-01 -1.12752879e+00 -1.01976120e+00 -3.04550433e+00
  -2.81003809e+00  5.76405108e-01  3.15406322e-01 -8.76941323e-01
  -2.93194437e+00 -8.59739721e-01 -1.49605310e+00 -1.95430684e+00
  -1.10958552e+00 -2.46726662e-01 -1.38538098e+00 -3.52531695e+00
  -2.60276818e+00  2.61455864e-01 -1.27252436e+00 -3.14690757e+00
  -7.45397806e-01 -7.04672754e-01  4.96683502e+00 -3.44809985e+00
   5.06579340e-01 -1.44525409e+00 -1.32578778e+00 -1.86176628e-01
  -3.12247300e+00  1.33025408e+00  6.12875986e+00 -1.16184640e+00
  -1.74448049e+00  4.60964292e-01  1.55875683e+00  4.49048318e-02
   6.61236286e+00 -3.00107908e+00 -2.77307320e

In [ ]:
def generate_text(model, start_string, size=1000):
  # Evaluation step (generating text using the learned model)
    print(start_string)
    num_generate = size

    # Convert our start_string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    #print(input_eval.shape)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0
    
    # make the batch...
    input_eval_64 = input_eval
    for i in range(63):
        input_eval_64 = np.vstack((input_eval_64,input_eval))
    input_eval = input_eval_64

    # Here batch size == 64
    model.reset_states()
    for i in range(num_generate):
        print("====================================================")
        print('input_eval',input_eval[0])
        predictions = model(input_eval)
        full_predictions = predictions
        print('full_predictions shape',full_predictions.shape)
        
        predictions = predictions[0]
        print('predictions shape',predictions.shape)
        print('predictions',to_text(predictions.numpy().argmax(axis=1)))
        # remove the batch dimension
        #predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        x = tf.random.categorical(predictions, num_samples=10)
        print(x.shape)
        print(x)
        for j in range(x.shape[1]):
              print(to_text(x[:,j].numpy()))
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        print('predicted_id',predicted_id)
        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        
        input_eval = tf.expand_dims([predicted_id], 0)
        print(input_eval)
        input_eval_64 = input_eval
        for i in range(63):
            input_eval_64 = np.vstack((input_eval_64,input_eval))
        input_eval = input_eval_64
  
        text_generated.append(idx2char[predicted_id])
        print('text so far',text_generated)
    return (start_string + ''.join(text_generated))

txt = generate_text(model, start_string="Polsko ",size=500)

In [17]:
# fin = open('pan_tadeusz.txt', 'rb')
# dataset_txt = fin.read().decode(encoding='utf-8')
# fin.close()

# # Obtain the unique characters
# vocab = sorted(set(dataset_txt))
# print ('{} unique characters'.format(len(vocab)))

# # Create a mapping from unique characters to indices
# char2idx = {char:index for index, char in enumerate(vocab)}
# print('char2idx:\n',char2idx)
# idx2char = np.array(vocab)
# print('idx2char\n',idx2char)
# vocab_size = len(vocab)

In [28]:
char2idx = {'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, ',': 5, '-': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20, 'K': 21, 'L': 22, 'M': 23, 'N': 24, 'O': 25, 'P': 26, 'R': 27, 'S': 28, 'T': 29, 'U': 30, 'V': 31, 'W': 32, 'Z': 33, 'a': 34, 'b': 35, 'c': 36, 'd': 37, 'e': 38, 'f': 39, 'g': 40, 'h': 41, 'i': 42, 'j': 43, 'k': 44, 'l': 45, 'm': 46, 'n': 47, 'o': 48, 'p': 49, 'q': 50, 'r': 51, 's': 52, 't': 53, 'u': 54, 'v': 55, 'w': 56, 'x': 57, 'y': 58, 'z': 59, 'Ó': 60, 'à': 61, 'é': 62, 'ó': 63, 'ą': 64, 'Ć': 65, 'ć': 66, 'ę': 67, 'Ł': 68, 'ł': 69, 'ń': 70, 'Ś': 71, 'ś': 72, 'Ź': 73, 'ź': 74, 'Ż': 75, 'ż': 76, '—': 77, '’': 78, '“': 79, '„': 80}
idx2char = ['\n',' ','!','(',')',',','-','.',':',';','?','A','B','C','D','E','F','G',
 'H','I','J','K','L','M','N','O','P','R','S','T','U','V','W','Z','a','b',
 'c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t',
 'u','v','w','x','y','z','Ó','à','é','ó','ą','Ć','ć','ę','Ł','ł','ń','Ś',
 'ś','Ź','ź','Ż','ż','—','’','“','„']

In [5]:
def to_text(sample):
  return ''.join([idx2char[int(x)] for x in sample])